<div class="alert alert-block alert-success">
    
# FIT5196 Task 1 in Assessment 1
#### Student Name: xxxx
#### Student ID: xxxx

Date: xxxx


Environment: Python xxxx

Libraries used:
* re (for regular expression, installed and imported) 
* pandas (for data manipulation) 
    
</div>

<div class="alert alert-block alert-danger">
    
## Table of Contents

</div>    

[1. Introduction](#Intro) <br>
[2. Importing Libraries](#libs) <br>
[3. Examining Patent Files](#examine) <br>
[4. Loading and Parsing Files](#load) <br>
$\;\;\;\;$[4.1. Defining Regular Expressions](#Reg_Exp) <br>
$\;\;\;\;$[4.2. Reading Files](#Read) <br>
$\;\;\;\;$[4.3. Whatever else](#latin) <br>
[5. Writing to CSV/JSON File](#write) <br>
$\;\;\;\;$[5.1. Verification - using the sample files](#test_xml) <br>
[6. Summary](#summary) <br>
[7. References](#Ref) <br>

-------------------------------------

<div class="alert alert-block alert-warning">

## 1.  Introduction  <a class="anchor" name="Intro"></a>
    
</div>

This assessment regards extracting data from semi-sctuctured text files. The dataset contained 500 `.txt` files which included various information about user reviews. In particular, ....

-------------------------------------

<div class="alert alert-block alert-warning">
    
## 2.  Importing Libraries  <a class="anchor" name="libs"></a>
 </div>

The packages to be used in this assessment are imported in the following. They are used to fulfill the following tasks:

* **re:** to define and use regular expressions
* **pandas:** ...
* ...

In [1]:

import re
import pandas

In [6]:
# from google.colab import drive
# drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

-------------------------------------

<div class="alert alert-block alert-warning">

## 3.  Examining Raw Data <a class="anchor" name="examine"></a>

 </div>

First of all, ....

Having examined the file content, the following observations were made:



-------------------------------------

<div class="alert alert-block alert-warning"> 

## 4.  Loading and Parsing Files <a class="anchor" name="load"></a>

</div>

In this section, the files are parsed and processed. First of all, appropriate regular expressions are defined to extract desired information when reading the files. ....

-------------------------------------

<div class="alert alert-block alert-info">
    
### 4.1. Defining Regular Expressions <a class="anchor" name="Reg_Exp"></a>

Defining correct regular expressions is crucial in extracting desired information from the text efficiently. ...

In [90]:
import pandas as pd

# 文件路径
text_path = './student_group181'

# 初始化一个空字符串，用于存储所有拼接的内容
all_text_data = ""

# 文件名列表
file_names = [
    'group181_0.txt', 'group181_1.txt', 'group181_2.txt', 'group181_3.txt', 
    'group181_4.txt', 'group181_5.txt', 'group181_6.txt', 'group181_7.txt', 
    'group181_8.txt', 'group181_9.txt', 'group181_10.txt', 'group181_11.txt',
    'group181_12.txt', 'group181_13.txt', 'group181_14.txt'
]

# 逐个读取每个文件
for file_name in file_names:
    file_path = f"{text_path}/{file_name}"
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        all_text_data += content  # 将内容拼接起来
        
split_txt = '?xml version="1.0" encoding="UTF-8"?>'
final_split= '</dataset><record>|<dataset><record>'

#通过split_txt，把内容切分成不同文件的部分
folder_text = all_text_data.split(split_txt)
# #删除第一个的空数值
if folder_text[0] == "":
    folder_text.pop(0)

#建立一个list，使用for循环便利folder_text，按照每个用户进行分割储存
final_split_text = []
for text in folder_text:
    slipted_info = re.split(final_split,text)
    final_split_text.extend(slipted_info)
    

In [104]:


user_id_expression = r"<[^>]*?ser[^>]*?>\s*(\d{21})\s*<[^>]*?/[^>]*?ser[^>]*?>"
# time_expression = r"((?:<.{1,3}ime>\s*)|(?:<.{0,2}[Dd]ate>\s*))(.*?)((?:<.{1,3}ime>)|(?:<.{0,2}[Dd]ate>))"

time_expression = r"(?:<.{1,3}ime\s*/?>\s*|<.{0,2}[Dd]ate\s*/?>\s*)(\d+)(?:\s*<\s*/\s*.{1,3}ime\s*/?>|<\s*/\s*.{0,2}[Dd]ate\s*/?>)"


#改了一些，可以在修改
response_expression = r"(?:<.{0,2}[Rr]esp.{0,4}>\s*)(.*?)(?:<.{0,2}[Rr]esp.{0,4}>)"
picture_expression = r"(?:<.{0,2}[Pp]ic.{1,5}>\s*)(.*?)(?:<.{0,2}[Pp]ic.{1,5}>)"

gmap_id_expression=r"(?:<.{0,2}[Gg]map.{1,3}>\s*)(.*?)(?:<.{0,2}[Gg]map.{1,3}>)"

In [107]:
# 测试0.txt数据
try_date = folder_text[0]
text_date = []
test_user_id_pattern = re.compile(r"<[^>]*?ser[^>]*?>\s*(\d{21})\s*<[^>]*?/[^>]*?ser[^>]*?>", re.DOTALL)
test_user_id_matches = test_user_id_pattern.findall(try_date)
text_date.extend(test_user_id_matches)
len(text_date)


4716

In [106]:
user_id = []
gmap_id = []
time = []
date = []

user_id_pattern = re.compile(user_id_expression, re.DOTALL)
gmap_id_pattern = re.compile(gmap_id_expression, re.DOTALL)
time_pattern = re.compile(time_expression, re.DOTALL)
# date_pattern = re.compile(date_expression, re.DOTALL)

#处理现在总的list数据，通过正则表达式提取需要的数据
for text in final_split_text:
     
     user_id_matches = user_id_pattern.findall(text)
     user_id.extend(user_id_matches)
     
     gmap_id_matches = gmap_id_pattern.findall(text)
     gmap_id.extend(gmap_id_matches)
     
     time_matches = time_pattern.findall(text)
     time.extend(time_matches)

     

['1622523469987',
 '1616313596636',
 '1618155881302',
 '1615822926221',
 '1612288803586',
 '1622515547819',
 '1620688412654',
 '1610335976198',
 '1617677596715',
 '1615593185281',
 '1621876717887',
 '1608268979149',
 '1611641455703',
 '1620190432197',
 '1622617639353',
 '1614035722318',
 '1618422300707',
 '1610573785647',
 '1614004565832',
 '1610474401031',
 '1599238071409',
 '1614697949346',
 '1601417085580',
 '1597541092127',
 '1603122452032',
 '1614569151254',
 '1508898740598',
 '1608075962039',
 '1587408897460',
 '1612745752963',
 '1609125063346',
 '1599344216633',
 '1602439157514',
 '1537197912453',
 '1603122729879',
 '1591666965631',
 '1612574857094',
 '1599857432407',
 '1590539389099',
 '1598037074152',
 '1609543172100',
 '1592625317661',
 '1619501039804',
 '1592872591088',
 '1592708550756',
 '1592401278806',
 '1581378969502',
 '1589163954130',
 '1589681644011',
 '1584402466034',
 '1590648666342',
 '1572141383038',
 '1590082868896',
 '1591719120945',
 '1594764723548',
 '15946088

In [72]:
attributes_pattern = re.compile("<.{1,10}>")
attributes = attributes_pattern.findall(all_text_data)

list_values = list(set(attributes))

date_attributes = []
for attr in list_values:
  if 'date' in attr.lower():
    date_attributes.append(attr)

# date_attributes


time_attributes = []
for attr in list_values:
  if 'time' in attr.lower():
    time_attributes.append(attr)

time_attributes


['<time>',
 '<//time>',
 '< /time>',
 '< time>',
 '<//Time>',
 '< Time>',
 '<Time>',
 '</Time>',
 '</time>',
 '< /Time>']

These patterns are used in the next step when reading the files.

-------------------------------------

<div class="alert alert-block alert-info">
    
### 4.2. Reading Files <a class="anchor" name="Read"></a>

In this step, all files are read and parsed. 

Let's take a look at the first ten elements of the lists generated. We can see that ids, reviews,etc. are parsed and stored correctly.

-------------------------------------

<div class="alert alert-block alert-info">
    
### 4.3. Whatever else <a class="anchor" name="latin"></a>

the rest of your methodology

-------------------------------------

<div class="alert alert-block alert-warning"> 

## 5.  Writing to JSON File <a class="anchor" name="write"></a>

</div>

....

-------------------------------------

<div class="alert alert-block alert-info">
    
### 5.1. Verification of the Generated JSON File <a class="anchor" name="test_xml"></a>

<div class="alert alert-block alert-warning"> 

## 6. Summary <a class="anchor" name="summary"></a>

</div>

....

-------------------------------------

....

-------------------------------------

<div class="alert alert-block alert-warning"> 

## 7. References <a class="anchor" name="Ref"></a>

</div>



[1]<a class="anchor" name="ref-2"></a> Why do I need to add DOTALL to python regular expression to match new line in raw string, https://stackoverflow.com/questions/22610247, Accessed 30/08/2022.

....


## --------------------------------------------------------------------------------------------------------------------------